In [29]:
import hashlib
import io
import logging
import os
import random
import re

import contextlib2
from lxml import etree
import numpy as np
import PIL.Image
import tensorflow as tf

from object_detection.dataset_tools import tf_record_creation_util
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util


In [30]:
#dict_to_tf_example
def dict_to_tf_example(filename,mask_path,label_map_dict,img_path):
    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    width = np.asarray(image).shape[1]
    height = np.asarray(image).shape[0]
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()
    
    

    with tf.gfile.GFile(mask_path, 'rb') as fid:
        encoded_mask_png = fid.read()
    encoded_png_io = io.BytesIO(encoded_mask_png)
    mask = PIL.Image.open(encoded_png_io)
    mask_np = np.asarray(mask.convert('L'))
    if mask.format != 'PNG':
        raise ValueError('Mask format not PNG')
        

    xmins = []
    ymins = []
    xmaxs = []
    ymaxs = []
    classes = []
    classes_text = []
    truncated = []
    poses = []
    difficult_obj = []
    masks = []
    

    class_name = 'nodule'
    nonbackground_indices_x = np.any(mask_np != 30, axis=0)
    nonbackground_indices_y = np.any(mask_np != 30, axis=1)
    nonzero_x_indices = np.where(nonbackground_indices_x)
    nonzero_y_indices = np.where(nonbackground_indices_y)

    if np.asarray(nonzero_x_indices).shape[1]>0 and np.asarray(nonzero_y_indices).shape[1]>0:
        xmin = float(np.min(nonzero_x_indices))
        xmax = float(np.max(nonzero_x_indices))
        ymin = float(np.min(nonzero_y_indices))
        ymax = float(np.max(nonzero_y_indices))
        print(filename, 'bounding box for', class_name,  xmin, xmax, ymin, ymax)

        xmins.append(xmin / width)
        ymins.append(ymin / height)
        xmaxs.append(xmax / width)
        ymaxs.append(ymax / height)

        classes_text.append(class_name.encode('utf8'))
        classes.append(label_map_dict[class_name])

        mask_remapped = (mask_np != 0).astype(np.uint8)
        masks.append(mask_remapped)

    feature_dict = {
          'image/height': dataset_util.int64_feature(height),
          'image/width': dataset_util.int64_feature(width),
          'image/filename': dataset_util.bytes_feature(
              filename.encode('utf8')),
          'image/source_id': dataset_util.bytes_feature(
              filename.encode('utf8')),
          'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
          'image/encoded': dataset_util.bytes_feature(encoded_jpg),
          'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
          'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
          'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
          'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
          'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
          'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
          'image/object/class/label': dataset_util.int64_list_feature(classes),
          'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
          'image/object/truncated': dataset_util.int64_list_feature(truncated),
          'image/object/view': dataset_util.bytes_list_feature(poses),
      }
    

    encoded_mask_png_list = []
    for mask in masks:
        img = PIL.Image.fromarray(mask)
        output = io.BytesIO()
        img.save(output, format='PNG')
        encoded_mask_png_list.append(output.getvalue())
    feature_dict['image/object/mask'] = (dataset_util.bytes_list_feature(encoded_mask_png_list))

    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example

In [31]:
#create_tf_record
def create_tf_record(output_filename,num_shards,label_map_dict,annotations_dir,image_dir,examples):
    writer = tf.io.TFRecordWriter(output_filename)
    for idx, example in enumerate(examples):
        if idx % 100 == 0:
            logging.info('On image %d of %d', idx, len(examples))
        mask_path = os.path.join(annotations_dir, example + '.png')
        image_path = os.path.join(image_dir, example + '.jpg')

        try:
            tf_example = dict_to_tf_example(example,
                                        mask_path,
                                        label_map_dict,
                                        image_path)
            if tf_example:
                writer.write(tf_example.SerializeToString())
                print("done")
        except ValueError:
            logging.warning('Invalid example: %s, ignoring.', xml_path)
    writer.close()


In [32]:
#main
data_dir = '/home/jtu9/lung/LUNA_data_train/'
train_output_path = '/home/jtu9/lung/LUNA_data_train/train.record'
image_dir = os.path.join(data_dir, 'JPEGImages')
annotations_dir = os.path.join(data_dir, 'Annotations/masks')
label_map_dict = label_map_util.get_label_map_dict('/home/jtu9/lung/LUNA_data_train/label.pbtxt')

In [33]:
#main
examples_list = os.listdir(image_dir) 
for el in examples_list:
    if el[-3:] !='jpg':
        del examples_list[examples_list.index(el)]
for el in examples_list:  
    examples_list[examples_list.index(el)] = el[0:-4]

In [34]:
#main_tobe
create_tf_record(train_output_path,1,label_map_dict,annotations_dir,image_dir,examples_list)

image1237 bounding box for nodule 472.0 489.0 147.0 171.0
done
image513 bounding box for nodule 144.0 196.0 304.0 360.0
done
image2073 bounding box for nodule 0.0 93.0 117.0 218.0
done
image2013 bounding box for nodule 155.0 170.0 336.0 355.0
done
image2150 bounding box for nodule 404.0 429.0 331.0 355.0
done
image2336 bounding box for nodule 371.0 401.0 263.0 293.0
done
image1670 bounding box for nodule 345.0 364.0 179.0 200.0
done
image47 bounding box for nodule 445.0 464.0 90.0 109.0
done
image1533 bounding box for nodule 59.0 74.0 112.0 129.0
done
image77 bounding box for nodule 90.0 117.0 266.0 300.0
done
image2722 bounding box for nodule 323.0 354.0 263.0 294.0
done
image1059 bounding box for nodule 97.0 121.0 42.0 68.0
done
image1548 bounding box for nodule 45.0 60.0 290.0 306.0
done
image1658 bounding box for nodule 57.0 82.0 352.0 377.0
done
image1841 bounding box for nodule 158.0 187.0 362.0 393.0
done
image2524 bounding box for nodule 382.0 416.0 308.0 344.0
done
image2677 b

image701 bounding box for nodule 163.0 200.0 28.0 64.0
done
image1542 bounding box for nodule 81.0 102.0 339.0 364.0
done
image2805 bounding box for nodule 383.0 417.0 308.0 344.0
done
image1566 bounding box for nodule 452.0 469.0 87.0 109.0
done
image1860 bounding box for nodule 48.0 63.0 84.0 100.0
done
image752 bounding box for nodule 435.0 446.0 323.0 334.0
done
image742 bounding box for nodule 10.0 37.0 120.0 150.0
done
image254 bounding box for nodule 63.0 86.0 56.0 80.0
done
image688 bounding box for nodule 302.0 329.0 304.0 337.0
done
image2311 bounding box for nodule 115.0 168.0 298.0 352.0
done
image220 bounding box for nodule 159.0 173.0 12.0 28.0
done
image2388 bounding box for nodule 325.0 384.0 298.0 357.0
done
image2708 bounding box for nodule 98.0 163.0 223.0 298.0
done
image304 bounding box for nodule 53.0 98.0 132.0 176.0
done
image1036 bounding box for nodule 427.0 480.0 265.0 318.0
done
image1527 bounding box for nodule 53.0 76.0 94.0 118.0
done
image204 bounding bo

image2358 bounding box for nodule 404.0 431.0 215.0 246.0
done
image1274 bounding box for nodule 95.0 111.0 43.0 59.0
done
image615 bounding box for nodule 333.0 446.0 58.0 171.0
done
image885 bounding box for nodule 14.0 82.0 116.0 185.0
done
image897 bounding box for nodule 116.0 145.0 278.0 311.0
done
image1963 bounding box for nodule 8.0 30.0 205.0 230.0
done
image1618 bounding box for nodule 162.0 193.0 318.0 348.0
done
image765 bounding box for nodule 118.0 184.0 267.0 334.0
done
image899 bounding box for nodule 451.0 468.0 165.0 182.0
done
image950 bounding box for nodule 94.0 187.0 0.0 86.0
done
image1956 bounding box for nodule 162.0 178.0 329.0 345.0
done
image2473 bounding box for nodule 286.0 324.0 46.0 90.0
done
image1840 bounding box for nodule 19.0 93.0 177.0 252.0
done
image667 bounding box for nodule 178.0 202.0 50.0 76.0
done
image726 bounding box for nodule 395.0 416.0 319.0 348.0
done
image796 bounding box for nodule 17.0 33.0 256.0 274.0
done
image2609 bounding box

image151 bounding box for nodule 131.0 142.0 16.0 28.0
done
image2090 bounding box for nodule 452.0 477.0 70.0 97.0
done
image406 bounding box for nodule 447.0 466.0 225.0 245.0
done
image2535 bounding box for nodule 435.0 460.0 148.0 173.0
done
image2100 bounding box for nodule 440.0 472.0 137.0 167.0
done
image2597 bounding box for nodule 366.0 414.0 139.0 188.0
done
image1435 bounding box for nodule 33.0 47.0 154.0 170.0
done
image857 bounding box for nodule 464.0 480.0 92.0 111.0
done
image450 bounding box for nodule 102.0 132.0 89.0 121.0
done
image2423 bounding box for nodule 78.0 98.0 73.0 93.0
done
image2397 bounding box for nodule 363.0 376.0 31.0 43.0
done
image644 bounding box for nodule 3.0 25.0 209.0 232.0
done
image1639 bounding box for nodule 157.0 229.0 96.0 174.0
done
image1365 bounding box for nodule 113.0 161.0 25.0 73.0
done
image884 bounding box for nodule 11.0 44.0 207.0 241.0
done
image2263 bounding box for nodule 437.0 460.0 156.0 181.0
done
image2368 bounding b

image1545 bounding box for nodule 441.0 474.0 213.0 248.0
done
image295 bounding box for nodule 48.0 69.0 67.0 92.0
done
image2683 bounding box for nodule 57.0 99.0 223.0 268.0
done
image1758 bounding box for nodule 474.0 495.0 130.0 151.0
done
image2236 bounding box for nodule 36.0 52.0 152.0 168.0
done
image2795 bounding box for nodule 90.0 137.0 308.0 359.0
done
image1511 bounding box for nodule 294.0 311.0 375.0 392.0
done
image2750 bounding box for nodule 328.0 360.0 234.0 270.0
done
image2560 bounding box for nodule 421.0 446.0 154.0 180.0
done
image2093 bounding box for nodule 104.0 130.0 293.0 320.0
done
image2670 bounding box for nodule 314.0 334.0 320.0 342.0
done
image393 bounding box for nodule 115.0 145.0 420.0 452.0
done
image1879 bounding box for nodule 175.0 227.0 261.0 313.0
done
image1387 bounding box for nodule 395.0 422.0 237.0 265.0
done
image2241 bounding box for nodule 376.0 397.0 261.0 282.0
done
image2575 bounding box for nodule 100.0 130.0 20.0 49.0
done
image

...

image232 bounding box for nodule 383.0 417.0 308.0 344.0
done
image2105 bounding box for nodule 453.0 476.0 313.0 336.0
done
image2227 bounding box for nodule 62.0 106.0 205.0 249.0
done
image739 bounding box for nodule 459.0 482.0 58.0 82.0
done
image879 bounding box for nodule 451.0 470.0 298.0 319.0
done
image1729 bounding box for nodule 208.0 230.0 160.0 183.0
done
image1411 bounding box for nodule 87.0 100.0 38.0 54.0
done
image2322 bounding box for nodule 391.0 409.0 23.0 41.0
done
image1775 bounding box for nodule 308.0 335.0 174.0 201.0
done
image780 bounding box for nodule 374.0 399.0 266.0 297.0
done
image2225 bounding box for nodule 393.0 449.0 240.0 300.0
done
image2118 bounding box for nodule 183.0 205.0 351.0 374.0
done
image2447 bounding box for nodule 40.0 59.0 221.0 242.0
done
image907 bounding box for nodule 38.0 64.0 195.0 221.0
done
image770 bounding box for nodule 123.0 171.0 119.0 199.0
done
image617 bounding box for nodule 385.0 461.0 155.0 231.0
done
image630 bo

image1120 bounding box for nodule 397.0 425.0 331.0 360.0
done
image2565 bounding box for nodule 452.0 469.0 85.0 106.0
done
image106 bounding box for nodule 487.0 508.0 193.0 218.0
done
image543 bounding box for nodule 254.0 280.0 167.0 197.0
done
image2354 bounding box for nodule 207.0 229.0 161.0 185.0
done
image771 bounding box for nodule 413.0 440.0 179.0 204.0
done
image894 bounding box for nodule 214.0 237.0 276.0 298.0
done
image1847 bounding box for nodule 401.0 423.0 101.0 123.0
done
image492 bounding box for nodule 309.0 331.0 291.0 312.0
done
done
image2328 bounding box for nodule 140.0 168.0 251.0 279.0
done
image1123 bounding box for nodule 97.0 130.0 282.0 313.0
done
image1173 bounding box for nodule 327.0 344.0 297.0 315.0
done
image1564 bounding box for nodule 127.0 162.0 230.0 266.0
done
image1702 bounding box for nodule 59.0 151.0 110.0 202.0
done
image494 bounding box for nodule 118.0 143.0 28.0 56.0
done
image1928 bounding box for nodule 16.0 34.0 163.0 182.0
done


image672 bounding box for nodule 366.0 388.0 379.0 403.0
done
image2602 bounding box for nodule 384.0 406.0 57.0 79.0
done
image379 bounding box for nodule 424.0 450.0 118.0 144.0
done
image1974 bounding box for nodule 136.0 188.0 138.0 191.0
done
image964 bounding box for nodule 75.0 88.0 57.0 71.0
done
image565 bounding box for nodule 122.0 170.0 338.0 384.0
done
image2307 bounding box for nodule 36.0 57.0 202.0 222.0
done
image216 bounding box for nodule 374.0 399.0 249.0 281.0
done
image44 bounding box for nodule 436.0 462.0 47.0 74.0
done
image198 bounding box for nodule 287.0 309.0 250.0 272.0
done
image1032 bounding box for nodule 412.0 511.0 179.0 275.0
done
image2464 bounding box for nodule 294.0 322.0 245.0 274.0
done
image1393 bounding box for nodule 346.0 373.0 56.0 84.0
done
image2628 bounding box for nodule 145.0 209.0 25.0 88.0
done
image1799 bounding box for nodule 459.0 478.0 300.0 319.0
done
image2272 bounding box for nodule 95.0 119.0 199.0 225.0
done
image2392 bound

image1743 bounding box for nodule 350.0 405.0 121.0 186.0
done
image2752 bounding box for nodule 451.0 467.0 321.0 337.0
done
image20 bounding box for nodule 59.0 98.0 141.0 182.0
done
image21 bounding box for nodule 321.0 383.0 9.0 75.0
done
image2413 bounding box for nodule 344.0 382.0 241.0 279.0
done
image2106 bounding box for nodule 58.0 73.0 125.0 140.0
done
image1600 bounding box for nodule 75.0 143.0 268.0 334.0
done
image2432 bounding box for nodule 436.0 480.0 126.0 170.0
done
image564 bounding box for nodule 420.0 465.0 237.0 282.0
done
image1015 bounding box for nodule 26.0 42.0 101.0 118.0
done
image1862 bounding box for nodule 54.0 69.0 256.0 271.0
done
image2033 bounding box for nodule 345.0 367.0 236.0 260.0
done
image59 bounding box for nodule 59.0 79.0 79.0 101.0
done
image2698 bounding box for nodule 112.0 136.0 360.0 386.0
done
image1033 bounding box for nodule 45.0 101.0 175.0 232.0
done
image5 bounding box for nodule 139.0 229.0 120.0 225.0
done
image315 bounding 

done
image1628 bounding box for nodule 53.0 104.0 119.0 172.0
done
image1300 bounding box for nodule 122.0 146.0 367.0 391.0
done
image1491 bounding box for nodule 55.0 73.0 353.0 373.0
done
image252 bounding box for nodule 178.0 196.0 273.0 295.0
done
image2430 bounding box for nodule 333.0 350.0 306.0 324.0
done
image589 bounding box for nodule 51.0 116.0 276.0 343.0
done
image855 bounding box for nodule 11.0 80.0 113.0 181.0
done
image1177 bounding box for nodule 469.0 488.0 209.0 229.0
done
done
image525 bounding box for nodule 331.0 357.0 387.0 413.0
done
image1311 bounding box for nodule 401.0 487.0 148.0 256.0
done
image887 bounding box for nodule 395.0 416.0 57.0 78.0
done
image1781 bounding box for nodule 466.0 504.0 174.0 214.0
done
image1961 bounding box for nodule 395.0 418.0 356.0 379.0
done
image101 bounding box for nodule 110.0 133.0 285.0 306.0
done
image658 bounding box for nodule 74.0 95.0 87.0 108.0
done
image2542 bounding box for nodule 162.0 176.0 218.0 234.0
done
